In [3]:
import pandas as pd
import numpy as np

# Simple Movie recommender

In [4]:
movies_df = pd. read_csv('datasets/movies_metadata.csv')
movies_df.head()

c:\users\ameya gupta\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
movies_df.shape

(45466, 24)

In [6]:
C = movies_df['vote_average'].mean() #mean imdb score
m = movies_df['vote_count'].quantile(0.80) #minimum vote count(top 20 percentile)

In [7]:
#just making a copy of original data frame with top 20 percentile movies(more votes than 80% of other movies)
movies_df2 = movies_df.copy().loc[movies_df['vote_count'] >= m]

In [8]:
print(C,m)

5.618207215134185 50.0


In [9]:
# calcuting score
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [10]:
# Adding a score column
movies_df2['score'] = movies_df2.apply(weighted_rating , axis = 1)

In [11]:
movies_df2 = movies_df2.sort_values('score', ascending=False)

# Top 15 predictions

In [12]:
movies_df2[['original_title', 'vote_count', 'vote_average', 'score']].head(15)

,original_title,vote_count,vote_average,score
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.855148
314,The Shawshank Redemption,8358.0,8.5,8.482863
834,The Godfather,6024.0,8.5,8.476278
40251,君の名は。,1030.0,8.5,8.366584
12481,The Dark Knight,12269.0,8.3,8.289115
2843,Fight Club,9678.0,8.3,8.286216
292,Pulp Fiction,8670.0,8.3,8.284623
522,Schindler's List,4436.0,8.3,8.270109
23673,Whiplash,4376.0,8.3,8.269704
5481,千と千尋の神隠し,3968.0,8.3,8.266628


# Genre based selection of top movies

In [13]:
#converting into pandas series(1-D) and stacking the same genres together
s = movies_df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
#joining movies dataset with s containing id and genres
gen_moviesdf = movies_df.drop('genres', axis=1).join(s)

In [14]:
#type(gen_moviesdf['genre'][0].find('Adventure'))
#df = gen_moviesdf[gen_moviesdf['budget'] == -1]
#df.loc[len(df.index)] = gen_moviesdf.iloc[10].to_list()
#df.reset_index(level=0, drop=True)
#print(df)
#counter = 0
#for i in range(len(gen_moviesdf['genre'])):
#    if(gen_moviesdf['genre'][i].find('Action') == -1):
#        continue
#    df.loc[counter] = gen_moviesdf.iloc[i].to_list()
#    counter += 1
#print(df)

In [15]:
### getting the df with row containg that particular movie rest is same as before using imdb rating formula
def get_movie(genre, percentile=0.85):
    df = gen_moviesdf[gen_moviesdf['budget'] == -1]
    counter = 0
    for i in range(len(gen_moviesdf['genre'])):
        if(gen_moviesdf['genre'][i].find(genre) == -1):
            continue
        df.loc[counter] = gen_moviesdf.iloc[i].to_list()
        counter += 1
    C = df['vote_average'].mean()
    m = df['vote_count'].quantile(percentile)
    df2 = df[df['vote_count'] >= m][['original_title', 'vote_count', 'vote_average','popularity']]
    
    # Calculation based on the IMDB formula
    df2['score'] = df2.apply(lambda x:(x['vote_count']/(x['vote_count']+m)*x['vote_average'])
                             +(m/(m+x['vote_count'])*C),axis=1)
    df2 = df2.sort_values('score', ascending=False)
    return df2

# Top 15 action movies

In [16]:
print(get_movie('Romance').head(15))

c:\users\ameya gupta\miniconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


                             original_title  vote_count  vote_average  \
2308            Dilwale Dulhania Le Jayenge       661.0           9.1   
6155                                  君の名は。      1030.0           8.5   
90                             Forrest Gump      8147.0           8.2   
283                   Nuovo Cinema Paradiso       834.0           8.2   
6223                             La La Land      4745.0           7.9   
4161                                    Her      4215.0           7.9   
1629  Eternal Sunshine of the Spotless Mind      3758.0           7.9   
213                                 Vertigo      1162.0           8.0   
701                             City Lights       444.0           8.2   
3270                             Mr. Nobody      1616.0           7.9   
6027                      Captain Fantastic      1569.0           7.9   
217                        Some Like It Hot       835.0           8.0   
219                              Casablanca      14

# Content based recommender

In [17]:
links_small = pd.read_csv('datasets/links_small.csv')
links_small = links_small['tmdbId']
links_small

0          862.0
1         8844.0
2        15602.0
3        31357.0
4        11862.0
          ...   
9120    402672.0
9121    315011.0
9122    391698.0
9123    137608.0
9124    410803.0
Name: tmdbId, Length: 9125, dtype: float64

In [18]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [19]:
movies_df['id'] = movies_df['id'].apply(convert_int)

In [20]:
movies_df[movies_df['id'].isnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",NaN,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",NaN,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
movies_df = movies_df.drop([19730, 29503, 35587])
smallmovies_df = movies_df[movies_df['id'].isin(links_small)]
smallmovies_df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844.0,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602.0,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357.0,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862.0,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40224,False,NaN,15000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,315011.0,tt4262980,ja,シン・ゴジラ,From the mind behind Evangelion comes a hit la...,...,2016-07-29,77000000.0,120.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,A god incarnate. A city doomed.,Shin Godzilla,False,6.6,152.0
40503,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}, {'id': 104...",http://www.thebeatlesliveproject.com/,391698.0,tt2531318,en,The Beatles: Eight Days a Week - The Touring Y...,"The band stormed Europe in 1963, and, in 1964,...",...,2016-09-15,0.0,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The band you know. The story you don't.,The Beatles: Eight Days a Week - The Touring Y...,False,7.6,92.0
44821,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",16000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://movies.warnerbros.com/pk3/,10991.0,tt0235679,ja,Pokémon 3: The Movie,When Molly Hale's sadness of her father's disa...,...,2000-07-08,68411275.0,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,False,6.0,144.0
44826,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://www.pokemon.com/us/movies/movie-pokemon...,12600.0,tt0287635,ja,劇場版ポケットモンスター セレビィ 時を越えた遭遇（であい）,"All your favorite Pokémon characters are back,...",...,2001-07-06,28023563.0,75.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,False,5.7,82.0


# Movie description(overview+taglines) based recommender

In [22]:
smallmovies_df['tagline'] = smallmovies_df['tagline'].fillna('')
smallmovies_df['description'] = smallmovies_df['overview'] + smallmovies_df['tagline']
smallmovies_df['description'] = smallmovies_df['description'].fillna('')

<ipython-input-22-066f7d8899e3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallmovies_df['tagline'] = smallmovies_df['tagline'].fillna('')
<ipython-input-22-066f7d8899e3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallmovies_df['description'] = smallmovies_df['overview'] + smallmovies_df['tagline']
<ipython-input-22-066f7d8899e3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [23]:
#smallmovies_df.info()

# TF-IDF

In [25]:
#youtube link for tf-idf : https://www.youtube.com/watch?v=D2V1okCEsiE
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smallmovies_df['description'])

In [26]:
tfidf_matrix.shape #longest description has 268124 words

(9099, 268124)

# Cosine similarity

In [27]:
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

In [28]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
#cs = np.dot(tfidf_matrix,tfidf_matrix)

In [29]:
cosine_sim.shape

(9099, 9099)

In [30]:
smallmovies_df = smallmovies_df.reset_index()
titles = smallmovies_df['title']
indices = pd.Series(smallmovies_df.index, index=smallmovies_df['title'])

In [31]:
#cosine_sim is a pairvise similarity matrix for movies and considering the top 30 ones for each title
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31] #taking first 30 top scores 
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [32]:
get_recommendations('The Godfather').head(10) #movies related to god father

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [33]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

# Metadata based recommender

In [34]:
ml = []
for chunk in  pd.read_csv('datasets/credits.csv',sep=',',chunksize= 20000):
    ml.append(chunk)

credits = pd.concat(ml, axis= 0)
del ml

In [35]:
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [36]:
keywords = pd.read_csv('datasets/keywords.csv')
keywords

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [37]:
#keywords['id'] = keywords['id'].astype('int')
#credits['id'] = credits['id'].astype('int')
#md['id'] = md['id'].astype('int')

In [38]:
movies_df = movies_df.merge(credits, on='id')
movies_df = movies_df.merge(keywords, on='id')

In [39]:
smallmovies_df = movies_df[movies_df['id'].isin(links_small)]
smallmovies_df.shape

(9219, 27)

In [40]:
from ast import literal_eval
smallmovies_df['cast'] = smallmovies_df['cast'].apply(literal_eval) #literal eval to convert string dictionary to dictionary
smallmovies_df['crew'] = smallmovies_df['crew'].apply(literal_eval) 
smallmovies_df['keywords'] = smallmovies_df['keywords'].apply(literal_eval)
smallmovies_df['cast_size'] = smallmovies_df['cast'].apply(lambda x: len(x))
smallmovies_df['crew_size'] = smallmovies_df['crew'].apply(lambda x: len(x))

<ipython-input-40-7b7dad4911fa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallmovies_df['cast'] = smallmovies_df['cast'].apply(literal_eval) #literal eval to convert string dictionary to dictionary
<ipython-input-40-7b7dad4911fa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallmovies_df['crew'] = smallmovies_df['crew'].apply(literal_eval)
<ipython-input-40-7b7dad4911fa>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [41]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [42]:
#for i in (smallmovies_df['crew'][0]):
#    print(i)

In [43]:
smallmovies_df['director'] = smallmovies_df['crew'].apply(get_director)

<ipython-input-43-a3799fe2e191>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallmovies_df['director'] = smallmovies_df['crew'].apply(get_director)


In [44]:
smallmovies_df['cast'] = smallmovies_df['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smallmovies_df['cast'] = smallmovies_df['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

<ipython-input-44-b277148890d3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallmovies_df['cast'] = smallmovies_df['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
<ipython-input-44-b277148890d3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallmovies_df['cast'] = smallmovies_df['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)


In [45]:
smallmovies_df['keywords'] = smallmovies_df['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

<ipython-input-45-1ed76c722889>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallmovies_df['keywords'] = smallmovies_df['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


In [46]:
smallmovies_df['cast'] = smallmovies_df['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

<ipython-input-46-4a19f8e83bc1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallmovies_df['cast'] = smallmovies_df['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])


In [47]:
smallmovies_df['director'] = smallmovies_df['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smallmovies_df['director'] = smallmovies_df['director'].apply(lambda x: [x,x])

<ipython-input-47-4307a33cf2b5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallmovies_df['director'] = smallmovies_df['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
<ipython-input-47-4307a33cf2b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallmovies_df['director'] = smallmovies_df['director'].apply(lambda x: [x,x])


In [48]:
s = smallmovies_df.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

<ipython-input-48-7894a5792596>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = smallmovies_df.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)


In [49]:
#counting the frequency of each keyword
s = s.value_counts()
s

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
                       ... 
town meeting              1
family farm               1
post holocaust            1
christmas lights          1
forced suicide            1
Name: keyword, Length: 12940, dtype: int64

In [50]:
s = s[s>1]

In [51]:
#stemming is a text processing part which is used to remove stems of a word may result in meaningless word
#lemmetization is used when meaning is need to be retained
from nltk.stem.snowball import SnowballStemmer #if nltk is not working in jn -https://stackoverflow.com/questions/61150183/cannot-import-nltk-in-jupyter-even-though-i-can-import-it-in-python-console
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [52]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [53]:
smallmovies_df['keywords'] = smallmovies_df['keywords'].apply(filter_keywords)
smallmovies_df['keywords'] = smallmovies_df['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smallmovies_df['keywords'] = smallmovies_df['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

<ipython-input-53-498cc3ec5b2d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallmovies_df['keywords'] = smallmovies_df['keywords'].apply(filter_keywords)
<ipython-input-53-498cc3ec5b2d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallmovies_df['keywords'] = smallmovies_df['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
<ipython-input-53-498cc3ec5b2d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [61]:
print(smallmovies_df)

       adult                              belongs_to_collection    budget  \
0      False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1      False                                                NaN  65000000   
2      False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3      False                                                NaN  16000000   
4      False  {'id': 96871, 'name': 'Father of the Bride Col...         0   
...      ...                                                ...       ...   
40952  False                                                NaN   8000000   
41172  False                                                NaN   1000000   
41225  False                                                NaN  15050000   
41391  False                                                NaN  15000000   
41669  False                                                NaN         0   

                                                  genres  \
0      [{'id': 

In [72]:
def convert(x):
    return list(x)
smallmovies_df['genres'] = smallmovies_df['genres'].apply(convert)


<ipython-input-72-5de58cbdfe79>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallmovies_df['genres'] = smallmovies_df['genres'].apply(convert)


In [75]:
#cols = ['keywords', 'cast', 'director','genres']
#smallmovies_df["combined"] = smallmovies_df["keywords"].str.cat(smallmovies_df[["cast", "director","genres"]].astype(str)
#                                                                , sep="_")
smallmovies_df['dump'] = smallmovies_df['keywords'] + smallmovies_df['cast'] + smallmovies_df['director'] + smallmovies_df['genres']
smallmovies_df['dump'] = smallmovies_df['dump'].apply(lambda x: ' '.join(x))    

<ipython-input-75-7cd22c652733>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallmovies_df['dump'] = smallmovies_df['keywords'] + smallmovies_df['cast'] + smallmovies_df['director'] + smallmovies_df['genres']
<ipython-input-75-7cd22c652733>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smallmovies_df['dump'] = smallmovies_df['dump'].apply(lambda x: ' '.join(x))


In [73]:
#type(smallmovies_df['genres'][0])

list

In [77]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smallmovies_df['dump'])

In [78]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [79]:
smallmovies_df = smallmovies_df.reset_index()
titles = smallmovies_df['title']
indices = pd.Series(smallmovies_df.index, index=smallmovies_df['title'])

In [80]:
#cosine similarities are changed so we can use same recommendation func to recommend
get_recommendations('The Dark Knight').head(10)

8031                   The Dark Knight Rises
6218                           Batman Begins
7659              Batman: Under the Red Hood
1134                          Batman Returns
6623                            The Prestige
9024      Batman v Superman: Dawn of Justice
2131                                Superman
8478    Justice League: Crisis on Two Earths
7648                               Inception
8001                        Batman: Year One
Name: title, dtype: object

# Popularity and ratings 

In [87]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26] #top 25movies based on similarity scores
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smallmovies_df.iloc[movie_indices][['title', 'vote_count', 'vote_average']] #removing useless features
    vote_counts = movies[movies_df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies_df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60) #60 percentile movies
    #imdb score to calculate weighted rating
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) 
                          & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [88]:
improved_recommendations('The Dark Knight')

<ipython-input-87-80e0bff93a75>:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vote_counts = movies[movies_df['vote_count'].notnull()]['vote_count'].astype('int')
<ipython-input-87-80e0bff93a75>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vote_averages = movies[movies_df['vote_average'].notnull()]['vote_average'].astype('int')
<ipython-input-87-80e0bff93a75>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_count'] = qualified['vote_count'].astype('int')
<ipython-input-87-80e0bff93a75>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,title,vote_count,vote_average,wr
7648,Inception,14075,8,7.991569
8613,Interstellar,11187,8,7.989402
6623,The Prestige,4510,8,7.973884
3381,Memento,4168,8,7.971766
8031,The Dark Knight Rises,9263,7,6.992581
6218,Batman Begins,7511,7,6.990862
8872,Captain America: Civil War,7462,7,6.990803
8419,Man of Steel,6462,6,5.997069
8467,Kick-Ass 2,2275,6,5.991789
9024,Batman v Superman: Dawn of Justice,7189,5,5.004270


# Collaborative Filtering

In [92]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [94]:
ratings = pd.read_csv('datasets/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [97]:
reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
#data.split(n_folds=5)

In [98]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5)#cv is folds

{'test_rmse': array([0.90535528, 0.89638674, 0.89719917, 0.89916459, 0.88617179]),
 'test_mae': array([0.69524761, 0.68951865, 0.693176  , 0.69107116, 0.68428101]),
 'fit_time': (4.6433587074279785,
  4.4534525871276855,
  4.5614097118377686,
  4.580345869064331,
  4.527974367141724),
 'test_time': (0.12497067451477051,
  0.22215771675109863,
  0.13367795944213867,
  0.1936485767364502,
  0.13563919067382812)}

In [101]:
#rmse is approximately 0.898 which is good enough
trainset = data.build_full_trainset()
svd.fit(trainset)

In [102]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [103]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.6770836132242666, details={'was_impossible': False})

# Hybrid recommender

In [104]:
id_map = pd.read_csv('datasets/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smallmovies_df[['title', 'id']], on='id').set_index('title')
#id_map = id_map.set_index('tmdbId')

In [105]:
indices_map = id_map.set_index('id')

In [109]:
#calculating cosine similarity first(content) and then ajusting acording to particular user(collaborative)
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smallmovies_df.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [110]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,id,est
1011,The Terminator,4208.0,7.4,218.0,3.358782
987,Alien,4564.0,7.9,348.0,3.201713
522,Terminator 2: Judgment Day,4274.0,7.7,280.0,3.176826
974,Aliens,3282.0,7.7,679.0,3.142986
8934,Home,1539.0,6.8,228161.0,3.052703
4987,Battle Royale,992.0,7.3,3176.0,3.050676
8401,Star Trek Into Darkness,4479.0,7.4,54138.0,2.922095
899,Platoon,1236.0,7.5,792.0,2.878213
4260,The Four Feathers,164.0,6.5,9093.0,2.842879
2014,Fantastic Planet,140.0,7.6,16306.0,2.827526


In [111]:
hybrid(1000, 'Avatar')

,title,vote_count,vote_average,id,est
1011,The Terminator,4208.0,7.4,218.0,3.954328
899,Platoon,1236.0,7.5,792.0,3.944161
974,Aliens,3282.0,7.7,679.0,3.918095
522,Terminator 2: Judgment Day,4274.0,7.7,280.0,3.914490
987,Alien,4564.0,7.9,348.0,3.913378
4987,Battle Royale,992.0,7.3,3176.0,3.904154
8401,Star Trek Into Darkness,4479.0,7.4,54138.0,3.854377
2014,Fantastic Planet,140.0,7.6,16306.0,3.763687
922,The Abyss,822.0,7.1,2756.0,3.641254
8231,Side by Side,111.0,7.1,110354.0,3.626591
